In [ ]:
import pandas as pd
from IPython.display import FileLink, display

## 1. Expand display settings in pandas SHOW ALL

In [ ]:
# Show ALL rows & columns, and don’t truncate text
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)  # <- key for full text
pd.set_option("display.width", None)

## 1. Expand display settings in pandas SHOW LIMITED

In [ ]:
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.max_colwidth")
pd.reset_option("display.width")
# pd.reset_option("all")

----------------------------------------------

## Check articleID

In [ ]:
import pandas as pd

# Read the CSV
# Eastview db
# df = pd.read_csv("soros_21092025_eastview.csv")
#----------------------------------
# Tsargard
df = pd.read_csv("tsargrad_soros_translated_21092025.csv")
# df = pd.read_csv("tsargrad_soros_translated_21092025_new.csv")
#----------------------------------
# Zavtra db
# df = pd.read_csv("zavtra_soros_translated_01092025.csv")
#----------------------------------


# Filter rows where score is empty (NaN or blank string)
missing_score = df[df["score"].isna() | (df["score"].astype(str).str.strip() == "")]

# Display the first few rows (with all headers)
# print(missing_score.head())


## How many records

In [ ]:
print("Total records:", len(df))

# If you also want to see how many are missing score:
missing_mask = df["score"].isna() | (df["score"].astype(str).str.strip() == "")
print("Records missing score:", missing_mask.sum())


In [ ]:
df.head()

## Eastview join

In [ ]:
import pandas as pd

def has_value(x):
    return (x is not None) and (not pd.isna(x)) and (str(x).strip().lower() not in {"", "nan", "none", "null"})

def norm_url(s):
    if pd.isna(s):
        return s
    return str(s).strip().rstrip("/")

def choose_best_text(row, text_cols):
    """
    Choose the 'best' text to use for ArticleTextEnglish when a score/prob exists.
    Strategy: pick the longest non-empty among ArticleTextEnglish, ArticleTextEnglish1..18.
    """
    best = None
    best_len = -1
    for c in text_cols:
        if c in row and has_value(row[c]):
            s = str(row[c]).strip()
            if len(s) > best_len:
                best = s
                best_len = len(s)
    return best

def merge_scores_with_texts(base_csv, scored_csv, out_csv, key="ArticleLink"):
    # 1) Load
    df = pd.read_csv(base_csv)
    dfu = pd.read_csv(scored_csv)

    # 2) Normalize key
    if key not in df.columns or key not in dfu.columns:
        raise KeyError(f"'{key}' must exist in both CSVs.")
    df[key] = df[key].apply(norm_url)
    dfu[key] = dfu[key].apply(norm_url)

    # 3) Keep only scored rows with usable values
    if "score" not in dfu.columns or "probability" not in dfu.columns:
        raise KeyError("Scored file must contain 'score' and 'probability' columns.")
    dfu = dfu[dfu["score"].apply(has_value) & dfu["probability"].apply(has_value)].copy()

    # 4) Identify auxiliary columns to bring over
    text_cols_all = ["ArticleTextEnglish"] + [f"ArticleTextEnglish{i}" for i in range(1, 19)]
    aux_candidates = text_cols_all + ["Term", "term", "word", "Word"]
    aux_cols_present = [c for c in aux_candidates if c in dfu.columns]

    # 5) If duplicates on key in scored file, keep highest probability
    dfu["probability_num"] = pd.to_numeric(dfu["probability"], errors="coerce")
    dfu = (dfu.sort_values("probability_num", ascending=False)
              .drop_duplicates(subset=[key], keep="first"))

    # 6) Build a 'best_text' in scored rows for updating ArticleTextEnglish
    dfu["_best_text"] = dfu.apply(lambda r: choose_best_text(r, text_cols_all), axis=1)

    # 7) Merge: pull score/prob + aux columns
    bring_cols = [key, "score", "probability", "_best_text"] + aux_cols_present
    merged = df.merge(dfu[bring_cols], on=key, how="left", suffixes=("", "_new"))

    # 8) Coalesce score/probability into base
    if "score_new" in merged.columns:
        merged["score"] = merged["score"].where(merged["score"].apply(has_value), merged["score_new"])
        merged.drop(columns=["score_new"], inplace=True)
    if "probability_new" in merged.columns:
        merged["probability"] = merged["probability"].where(merged["probability"].apply(has_value), merged["probability_new"])
        merged.drop(columns=["probability_new"], inplace=True)

    # 9) Update base ArticleTextEnglish WHEN a score/probability exists and we have a best_text
    #    (This "updates" the original text field as requested.)
    if "ArticleTextEnglish" not in merged.columns:
        merged["ArticleTextEnglish"] = pd.NA
    cond_has_scored = merged["score"].apply(has_value) & merged["probability"].apply(has_value)
    merged.loc[cond_has_scored & merged["_best_text"].apply(has_value), "ArticleTextEnglish"] = merged.loc[
        cond_has_scored, "_best_text"
    ]

    # 10) Ensure all aux columns exist in final output.
    # For any aux col:
    #   - if the base already has it, only fill missing values from the scored version (col_new)
    #   - if the base doesn't have it, create/rename from the scored version
    for c in aux_cols_present:
        new_col = f"{c}_new"
        if new_col in merged.columns and c in merged.columns:
            merged[c] = merged[c].where(merged[c].apply(has_value), merged[new_col])
            merged.drop(columns=[new_col], inplace=True)
        elif new_col in merged.columns and c not in merged.columns:
            merged.rename(columns={new_col: c}, inplace=True)

    # 11) Drop helper columns
    if "_best_text" in merged.columns:
        merged.drop(columns=["_best_text"], inplace=True)

    # 12) Optional: drop rows still missing score/probability (as per previous step)
    final = merged[merged["score"].apply(has_value) & merged["probability"].apply(has_value)].copy()

    # 13) Save
    final.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"Merged rows with scores: {final.shape[0]} / {merged.shape[0]}")
    print(f"Saved: {out_csv}")
    return final, merged



In [ ]:
# # Example: Zavtra pair (if you want to re-run with the same logic) ----
# final_zavtra, merged_zavtra = merge_scores_with_texts(
#     base_csv="zavtra_soros_translated_01092025.csv",
#     scored_csv="zavtra_soros_01102025_scored.csv",
#     out_csv="zavtra_soros_final.csv",
#     key="ArticleLink"
# )


In [ ]:

# ---- Example: Eastlink pair ----
final_eastlink, merged_eastlink = merge_scores_with_texts(
    base_csv="eastlink_soros_translated_01092025.csv",
    scored_csv="soros_translated_01102025.csv",
    out_csv="eastlink_soros_final.csv",
    key="ArticleLink"
)


## Add back terms

In [ ]:
import pandas as pd
import re

# --- config ---
in_csv  = "eastlink_soros_final.csv"
out_csv = "eastlink_soros_final_replaced.csv"   # change to same name if you want in-place
target_ids_raw = [
    "4158790","42260504","19335327","4280","3723631",
    "9367502","5838159","2188538","23528922"
]

# Load
df = pd.read_csv(in_csv)

# Normalise ArticleID so strings like "4280.0" match too
df["_ArticleID_int"] = pd.to_numeric(df["ArticleID"], errors="coerce").astype("Int64")
target_ids = pd.to_numeric(pd.Series(target_ids_raw), errors="coerce").astype("Int64")
mask_ids = df["_ArticleID_int"].isin(set(target_ids))

# Prepare counts (before)
pattern_ci = r"(?i)\bSoros\b"   # whole-word, case-insensitive
subset = df.loc[mask_ids, ["ArticleID", "ArticleTextEnglish"]].copy()
subset["count_before"] = subset["ArticleTextEnglish"].fillna("").str.count(pattern_ci)

# Do the replacement only for the targeted IDs
repl_pat = re.compile(r"\b[Ss]oros\b")  # same as (?i)\bSoros\b but lets us keep 'Soroses' case
col = "ArticleTextEnglish"
df.loc[mask_ids & df[col].notna(), col] = (
    df.loc[mask_ids & df[col].notna(), col]
      .str.replace(repl_pat, "Soroses", regex=True)
)

# Counts (after) for sanity check
subset["count_after"] = df.loc[mask_ids, "ArticleTextEnglish"].fillna("").str.count(pattern_ci)
subset["replacements"] = subset["count_before"] - subset["count_after"]

print("Total rows targeted:", mask_ids.sum())
print("Total 'Soros' tokens replaced:",
      int(subset["replacements"].clip(lower=0).sum()))

# Optional: inspect which IDs actually changed
changed = subset[subset["replacements"] > 0]
display(changed[["ArticleID","replacements"]].sort_values("replacements", ascending=False).head(20))

# Save
df.drop(columns=["_ArticleID_int"], inplace=True)
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("Saved:", out_csv)


In [ ]:
import pandas as pd
import os
import re

# Files to process
inputs = [
    "eastlink_soros_final_replaced2.csv",
    "zavtra_soros_final.csv",
]

# Use today's date in the filename (keep your project’s ddmmyyyy style)
date_suffix = "01102025"  # 01-10-2025

# Regex: whole-word, case-insensitive, for the listed variants
pattern = r"(?i)\b(?:soros|soroses|sorosyats)\b"

def add_soros_count(in_csv):
    df = pd.read_csv(in_csv)

    # find all ArticleTextEnglish* columns that exist
    text_cols = [c for c in df.columns if c.startswith("ArticleTextEnglish")]
    if not text_cols:
        print(f"[WARN] No ArticleTextEnglish* columns in {in_csv}")
        df["soros_count"] = 0
    else:
        # sum counts across all text columns, treating NaN as empty string
        counts = None
        for i, c in enumerate(text_cols):
            col_counts = df[c].fillna("").astype(str).str.count(pattern)
            counts = col_counts if counts is None else (counts + col_counts)

        df["soros_count"] = counts.astype(int)

    # build output path with date suffix
    root, ext = os.path.splitext(in_csv)
    out_csv = f"{root}_{date_suffix}{ext}"

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"Processed: {in_csv}  →  {out_csv}")
    print("  ArticleTextEnglish* columns:", text_cols)
    print("  Rows with soros_count > 0:", int((df['soros_count'] > 0).sum()))
    return out_csv

# Run for both files
outputs = [add_soros_count(p) for p in inputs]

# Quick peek (optional)
for out in outputs:
    df_check = pd.read_csv(out, nrows=5)
    display(df_check.head(3)[["soros_count"] + [c for c in df_check.columns if c.startswith("ArticleTextEnglish")][:2]])


In [ ]:
import pandas as pd
import re

# --- config ---
in_csv  = "eastlink_soros_final_replaced.csv"   # set to the file you're editing
out_csv = "eastlink_soros_final_replaced2.csv"  # set same as in_csv for in-place
target_id_raw = "10022567"

# Load
df = pd.read_csv(in_csv)

# Robust match on ArticleID
df["_ArticleID_int"] = pd.to_numeric(df["ArticleID"], errors="coerce").astype("Int64")
target_id = pd.to_numeric(target_id_raw, errors="coerce")
mask = df["_ArticleID_int"].eq(target_id)

# Count matches before (for sanity)
pattern_ci = r"(?i)\bSoros\b"
before = int(df.loc[mask, "ArticleTextEnglish"].fillna("").str.count(pattern_ci).sum())

# Replace only for the targeted row(s)
repl_pat = re.compile(r"\b[Ss]oros\b")
col = "ArticleTextEnglish"
df.loc[mask & df[col].notna(), col] = (
    df.loc[mask & df[col].notna(), col]
      .str.replace(repl_pat, "SOROSYATS", regex=True)
)

# Count remaining "Soros" tokens after
after = int(df.loc[mask, "ArticleTextEnglish"].fillna("").str.count(pattern_ci).sum())
print(f"Rows targeted: {mask.sum()} | Replacements made: {before - after}")

# (Optional) display the edited rows
display(df.loc[mask, ["ArticleID", col]].head())

# Save
df.drop(columns=["_ArticleID_int"], inplace=True)
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("Saved:", out_csv)


In [ ]:
import pandas as pd
import re

# --- config ---
in_csv  = "tsargrad_soros_translated_21092025.csv"
out_csv = "tsargrad_soros_translated_01102025_final.csv"

# Load
df = pd.read_csv(in_csv)

# Columns to search: all ArticleTextEnglish* plus common text fields in this dataset
text_cols = [c for c in df.columns if c.startswith("ArticleTextEnglish")]
text_cols += [c for c in [
    "ArticleTextEnglish",
] if c in df.columns]

# Deduplicate while preserving order
text_cols = list(dict.fromkeys(text_cols))
print("Counting in columns:", text_cols)

# Count whole-word 'soros' (case-insensitive)
pattern = re.compile(r"(?i)\bSoros\b")

if text_cols:
    counts = None
    for c in text_cols:
        col_counts = df[c].fillna("").astype(str).str.count(pattern)
        counts = col_counts if counts is None else (counts + col_counts)
    df["soros_count"] = counts.astype(int)
else:
    df["soros_count"] = 0

# Save
df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print(f"Saved: {out_csv}")
print("Rows with soros_count > 0:", int((df["soros_count"] > 0).sum()))


In [ ]:
df.head()

In [ ]:
# missing_score.to_csv("missing_score_tsargrad_21092025_2.csv", index=False)

In [ ]:
df.head()

## inspect IDs have missing score

In [ ]:

# rows with missing/blank score
missing_mask = df["score"].isna() | df["score"].astype(str).str.strip().isin(["", "nan", "NaN", "None", "null"])
missing = df.loc[missing_mask].copy()

print("Missing rows:", len(missing))  # should be 29

# save them
out_csv = "tsargrad_missing_score_21092025.csv"
missing.to_csv(out_csv, index=False, encoding="utf-8-sig")
display(FileLink(out_csv))  # clickable link in Jupyter

# (optional) Excel + just the IDs
# missing.to_excel("tsargrad_missing_score_21092025.xlsx", index=False)
# missing["ArticleID"].dropna().astype(str).to_csv("tsargrad_missing_ids_21092025.txt", index=False, header=False)


## Copy manual collected info from .csv to original .csv to update score/ article text google translator

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# ---- file paths (edit if needed) ----
old_path = Path("tsargrad_soros_translated_21092025.csv")
new_path = Path("tsargrad_soros_translated_21092025_new.csv")

# Columns to overwrite from _new -> old
cols_to_copy = [
    "title", "translated_title", "author", "article_text",
    "ArticleTextEnglish", "score", "probability"
]

# Read CSVs (adjust encoding if needed, e.g., encoding='utf-8-sig')
old_df = pd.read_csv(old_path)
new_df = pd.read_csv(new_path)

# Basic sanity checks
assert "url" in old_df.columns, "old CSV missing 'url' column"
assert "url" in new_df.columns, "new CSV missing 'url' column"

# Ensure URL uniqueness (warn but proceed by keeping first)
if new_df["url"].duplicated().any():
    print("WARNING: Duplicated URLs in NEW file; keeping the first occurrence.")
    new_df = new_df.drop_duplicates(subset="url", keep="first")
if old_df["url"].duplicated().any():
    print("WARNING: Duplicated URLs in OLD file; keeping the first occurrence for preview/diff only.")


In [ ]:
# Use URL as index to align
old_df_idx = old_df.set_index("url", drop=False)
new_df_idx = new_df.set_index("url", drop=False)

common_urls = old_df_idx.index.intersection(new_df_idx.index)

# Columns that exist in both
effective_cols = [c for c in cols_to_copy if c in old_df_idx.columns and c in new_df_idx.columns]

# Snapshot BEFORE update
before = old_df_idx.loc[common_urls, effective_cols].copy()

# Perform overwrite
old_df_idx.loc[common_urls, effective_cols] = new_df_idx.loc[common_urls, effective_cols].values

# Snapshot AFTER update
after = old_df_idx.loc[common_urls, effective_cols].copy()


In [ ]:
changed = (before != after) & ~(before.isna() & after.isna())

print("Number of changed cells:", changed.sum().sum())

# Show first 20 differences
diff_df = []
for col in effective_cols:
    mask = changed[col]
    if mask.any():
        sub = pd.DataFrame({
            "url": before.index[mask],
            "field": col,
            "old": before.loc[mask, col],
            "new": after.loc[mask, col],
        })
        diff_df.append(sub)

diff_df = pd.concat(diff_df, ignore_index=True)
display(diff_df.head(20))


In [ ]:
# Count how many rows actually changed in at least one column
rows_changed = (before != after).any(axis=1).sum()
print(f"Rows changed: {rows_changed}")

# Count how many columns changed per row
per_row_changes = (before != after).sum(axis=1)
print(per_row_changes.value_counts().sort_index())


In [ ]:
# save
from datetime import datetime

# Make a timestamped backup before overwriting
ts = datetime.now().strftime("%Y%m%d-%H%M%S")
backup_path = old_path.with_suffix(f".bak_{ts}.csv")

# Save the untouched original as backup
old_df.to_csv(backup_path, index=False)
print(f" Backup saved → {backup_path}")

# Save the updated dataframe (reset index to drop 'url' as index)
old_df_updated = old_df_idx.reset_index(drop=True)
old_df_updated.to_csv(old_path, index=False)
print(f" Updated CSV saved → {old_path}")

## Project Scope
Delete specific ArticleIDs

In [ ]:
# import pandas as pd

# # Load CSV
# df = pd.read_csv("soros_21092025_eastview.csv")

# # Make sure ArticleID is treated as string (avoids type mismatch issues)
# df["ArticleID"] = df["ArticleID"].astype(str).str.strip()

# # IDs you want to delete (as strings)
# to_remove = ["285237", "4961499", "4953787"]

# # Drop them
# df_cleaned = df[~df["ArticleID"].isin(to_remove)]

# # Verify — should be empty if removal worked
# print(df_cleaned[df_cleaned["ArticleID"].isin(to_remove)])

# # Check row counts
# print("Before:", len(df), "After:", len(df_cleaned))

# # Save back
# df_cleaned.to_csv("soros_21092025_eastview_cleaned.csv", index=False)


In [ ]:
import pandas as pd

df = pd.read_csv("soros_21092025_eastview.csv")

to_remove = {285237, 4961499, 4953787}
present = set(df["ArticleID"]).intersection(to_remove)

if present:
    print("Found and removing:", present)
    df = df[~df["ArticleID"].isin(present)]
    df.to_csv("soros_21092025_eastview_cleaned.csv", index=False)
    print("New row count:", len(df))
else:
    print("None of the target ArticleIDs are in this file:", to_remove)
    print("Sample of nearby IDs for context:")
    print(df["ArticleID"].head(20).to_list())


In [ ]:
import pandas as pd

# Read the cleaned CSV
df = pd.read_csv("zavtra_soros_translated_01092025.csv")

# Show the first few rows
df.head()

In [ ]:
df.columns

In [ ]:
# 1. Count total rows vs. non-null rows in 'score'
print("Total rows:", len(df))
print("Rows with score:", df['score'].notna().sum())

# 2. Check how many are missing
missing_scores = df['score'].isna().sum()
print("Missing score rows:", missing_scores)

# 3. If you want to actually see the missing rows
df[df['score'].isna()]

# 4. If score could be blank strings instead of NaN
blank_scores = (df['score'].astype(str).str.strip() == "").sum()
print("Blank score rows:", blank_scores)


In [ ]:
# How many missing per Publication
print(missing_df['Publication'].value_counts())

# How many missing per Year
print(missing_df['Year'].value_counts())

# Look at just the ArticleIDs
missing_ids = missing_df['ArticleID'].tolist()
print(missing_ids[:20])  # print first 20 IDs


In [ ]:
# Show all missing score rows in a table (scrollable in Jupyter)
missing_df = df[df['score'].isna()]
missing_df


In [ ]:
# Add a column with character count of the English text
missing_df = df[df['score'].isna()].copy()
missing_df['char_len'] = missing_df['ArticleTextEnglish'].astype(str).str.len()

# See distribution of lengths
print(missing_df['char_len'].describe())

# See the top 10 longest texts
print(missing_df[['ArticleID', 'Publication', 'Year', 'char_len']].sort_values(by='char_len', ascending=False).head(10))


In [ ]:
# Case-insensitive search for 'soros' in ArticleTextEnglish
mask = df['ArticleTextEnglish'].str.contains("soros", case=False, na=False)

# Get all rows where 'soros' appears
soros_df = df[mask]

# Show a few examples
soros_df[['ArticleID', 'Publication', 'Year', 'ArticleTextEnglish']].head()


In [ ]:
missing_df = df[df['score'].isna()].copy()
mask = missing_df['ArticleTextEnglish'].str.contains("soros", case=False, na=False)

soros_missing = missing_df[mask]

print("Missing rows with 'soros' in text:", len(soros_missing))
soros_missing[['ArticleID', 'Publication', 'Year', 'ArticleTextEnglish']].head()


In [ ]:
# Export all rows that mention 'soros'
soros_df.to_csv("soros_mentions.csv", index=False, encoding="utf-8")
print("Exported", len(soros_df), "rows to soros_mentions.csv")


In [ ]:
# Quick summary of text lengths
df["ArticleText"].str.len().describe()

# Filter for a particular ArticleID
df[df.ArticleID == "10012711"].T

# Count how many records per Publication Year
df["Publication Date"].dt.year.value_counts().sort_index()


In [ ]:
# create a boolean mask: True if ArticleText is non‑empty after stripping whitespace
has_text = df["ArticleText"].fillna("").str.strip() != ""

# count how many have text vs don’t
counts = has_text.value_counts().rename(index={True: "with_text", False: "without_text"})
print(counts)


### Check PNGS imported

In [ ]:
import pandas as pd

# 1. Load your CSV
df = pd.read_csv("soros_text.csv", dtype=str).fillna("")

# 2. Filter down to just the PNG‑based records
png_df = df[df["ZipFile"].str.lower().str.endswith(".png")]

# 3. Build a boolean mask: True if there’s any non‑whitespace in ArticleText
has_text = png_df["ArticleText"].str.strip() != ""

# 4. Print counts
print(f"Total PNGs processed      : {len(png_df)}")
print(f"PNGs with extracted text  : {has_text.sum()}")
print(f"PNGs without extracted text: {len(png_df) - has_text.sum()}")

# Show the first few that failed:
print("\nPNGs that produced NO text:")
print(png_df.loc[~has_text, ["ZipFile","ArticleText"]].head(), "\n")

# And a few that succeeded:
print("PNGs that DID extract some text:")
print(png_df.loc[has_text, ["ZipFile","ArticleText"]].head())



### Check png ids

In [ ]:
import pandas as pd

# 1) Load your cleaned CSV
df = pd.read_csv("soros_text.csv", dtype=str).fillna("")

# 2) List PNG IDs to check
ids = [
    "43450062","45780836","46081842","46155643","46186228","46237045",
    "46590726","46652831","46679825","46830363","46889551","47040605",
    "47834519","48128305","48191549","48210782","48280247","48765228",
    "48871713","49079885","49361014","49563408","49680285","49859217",
    "50939659","51225798","51337298","51901452","52320070","52818826",
    "53259695","53518917","53887820","56256564","56710275","59288998",
    "59459315"
]

# 3) Filter
subset = df[df["ArticleID"].isin(ids)]

# 4) How many have non‑empty ArticleText?
with_text    = subset["ArticleText"].str.strip().astype(bool).sum()
without_text = len(subset) - with_text
print(f"{len(subset)} total IDs\n→ with text   : {with_text}\n→ without text: {without_text}\n")

# 5) Peek at the ones missing text
print("IDs with missing text:")
print(subset[subset["ArticleText"].str.strip() == ""]["ArticleID"].tolist())

# 6) And sample of those that did extract:
print("\nSample extracted text:")
print(subset[subset["ArticleText"].str.strip() != ""][["ArticleID","ArticleText"]].head(3))


### Check PDFs imported

In [ ]:
# df = pd.read_csv("soros_text.csv", dtype=str).fillna("")
# ids = ["10012711", "10012758", "10021857", "10060474", "10110314", "10124118", "10130531", "10137", "10214979", "10228407", "10491551", "10493649", "10682772", "11147", "11204991", "11322392", "11376316", "11396980", "11400", "11407779", "11411652", "11614680", "1167", "11831910", "11875889", "11877061", "11911212", "11931073", "12075116", "12075586", "12247814", "12261076", "12261077", "12298431", "12723219", "12800283", "12908500", "12929645", "12957670", "12968446", "13068430", "13092748", "13255401", "13255434", "13276200", "14080180", "1447", "14746291", "14764054", "1750044729", "1750044756", "1750044780", "1750044801", "1750045011", "18530836", "18672929", "18761394", "18770939", "18981209", "18991913", "19032349", "19035059", "19124361", "19287542", "19359219", "19484195", "19612370", "19669286", "19726145", "19771679", "19826195", "19923542", "19953013", "19975642", "20098411", "20166113", "20268856", "20432991", "20460473", "20462359", "20515198", "20574709", "20658875", "20709184", "20785406", "20938540", "2105770", "2107390", "2107558", "2108346", "2109114", "2109753", "2110094", "21188636", "21250044", "21293652", "2130105", "2130824", "21347366", "2135847", "2137686", "2138649", "2138727", "21482574", "2157603", "2158028", "2162202", "2162539", "21625812", "21636781", "2164830", "2165502", "2168241", "2179890", "2187948", "2188180", "2189962", "2190096", "2190489", "2191171", "2191350", "2191374", "2191510", "2191633", "2191646", "2192593", "2192899", "2193067", "2193124", "2193893", "2194023", "2194037", "2194162", "2194222", "2194332", "2194957", "2195015", "2195112", "2195514", "2196401", "2196666", "2196844", "22568763", "22838932", "23255672", "2329758", "23300130", "2330607", "2331522", "23379764", "23457205", "2403543", "2403905", "2405780", "2407303", "2407832", "2409053", "2409098", "2409978", "2411229", "2411383", "2415134", "2415300", "2419018", "2424121", "2424207", "2424842", "2426314", "24278986", "2432957", "2434299", "2435371", "2437516", "2439774", "2440493", "2440966", "2441032", "2441403", "2442876", "2448810", "24861986", "25761140", "25916073", "26191336", "26239338", "26312899", "26345940", "26658188", "267898", "26850258", "27622039", "276480", "276769", "27698022", "278160", "279257", "279258", "279833", "28078435", "28108755", "28109218", "284502", "285237", "285540", "28595762", "28710929", "28844176", "29252278", "29454859", "29498634", "29607948", "298477", "29867190", "29888074", "29889084", "303065", "30373426", "303959", "304093", "305141", "305616", "305795", "305865", "306105", "306567", "307577", "308071", "308203", "309897", "309954", "310196", "310854", "312056", "315304", "316369", "317052", "317224", "317282", "318415", "319672", "320142", "320144", "321630", "322168", "322271", "322648", "323732", "325045", "327194", "327390", "327545", "332266", "3464154", "3464176", "3464631", "3465168", "3466536", "3466698", "3466899", "3467628", "3468090", "3469071", "3469674", "3469961", "3470168", "3471430", "3473306", "3473550", "34818350", "35244234", "35851437", "36000606", "36021200", "36058932", "36107825", "36107950", "36395681", "36413999", "36506561", "37269835", "37689266", "37689427", "37706855", "37719250", "37740682", "37834274", "37867818", "38623245", "3872132", "39128008", "3917019", "39191803", "39253550", "3953289", "4059170", "40678772", "40846261", "4095755", "41297084", "41467526", "41520943", "4158492", "4158790", "4206937", "42162547", "4217333", "42321440", "4237588", "42426863", "4245423", "42564330", "42580160", "42601175", "42740408", "42798891", "42820421", "42841428", "42844413", "42940705", "42953765", "42973236", "4310025", "4332384", "43335259", "4370561", "43864581", "4410099", "4428914", "44540655", "4457225", "44586218", "44783215", "44783233", "44809697", "4487", "44937489", "45085825", "45120173", "45163176", "45235282", "4534", "45353100", "45386197", "45489106", "45489108", "45705034", "45812104", "4582313", "45826120", "45975893", "45984198", "45991332", "46002287", "46022309", "46022310", "46024087", "46033685", "4606", "46188048", "46222890", "46237047", "46287407", "4637963", "46397855", "46399554", "4642491", "46431219", "46488755", "46497683", "46543824", "46721285", "46742677", "46760929", "46800290", "4681934", "46822146", "46859511", "4685983", "46863058", "46886399", "46889509", "4689176", "4689502", "47042265", "47082683", "47094466", "47174907", "47558736", "47670617", "47687945", "47716365", "47823928", "47885479", "48021687", "48021743", "48137043", "48189090", "48212888", "48259395", "4848531", "48575326", "48594769", "48609659", "48649178", "48650200", "48682296", "48789319", "48844470", "4889395", "48966866", "49297858", "49385985", "49564054", "49593638", "4969215", "49708713", "49807008", "49934268", "49954494", "50031193", "5014873", "50325487", "50412491", "5046237", "50828249", "50857949", "5097003", "5113870", "5117647", "5143163", "51614643", "51616512", "51839600", "51857978", "51946156", "51965969", "51972322", "52040572", "52073680", "52074383", "52289573", "52289614", "5229215", "5241135", "52458978", "52733125", "52757830", "53135485", "53174198", "53276473", "5332922", "5333984", "5359827", "53916190", "53927202", "54001466", "54848525", "54991897", "5569657", "5572489", "55934740", "56147896", "5632599", "56414434", "56604908", "5667822", "57192503", "5772044", "5782498", "5817081", "5826635", "5831454", "5838108", "5842454", "5842478", "58709113", "59267169", "5952566", "6053775", "6063326", "6079023", "6081302", "6081306", "60861681", "60932377", "6105391", "61486113", "61723954", "62051793", "6205543", "6223037", "62506412", "6306421", "63294008", "6507", "6521169", "6533255", "6603029", "6611882", "6656277", "68032659", "6909372", "6929", "7009", "7012403", "7049276", "7070299", "7151751", "7180777", "7210829", "7273512", "7282622", "7305", "7576248", "7681169", "7695433", "7747272", "7755691", "7812272", "78706085", "7901", "7915932", "7964028", "8161579", "8183791", "8212732", "8240787", "8441541", "8515751", "8588339", "8596692", "8730296", "8744257", "8749749", "8786106", "8860966", "9039690", "9124723", "9147024", "9184404", "9193869", "9208314", "9242", "9251", "9255771", "9320963", "9378997", "9392495", "9400", "94314565", "9523663", "9539796", "9542459", "9564985", "9606", "9667316", "9706592", "9726453", "9731173", "9737074", "9740249", "9850587", "9895356", ]

# subset = df[df["ArticleID"].isin(ids)]

# # 4) How many have non‑empty ArticleText?
# with_text    = subset["ArticleText"].str.strip().astype(bool).sum()
# without_text = len(subset) - with_text
# print(f"{len(subset)} total IDs\n→ with text   : {with_text}\n→ without text: {without_text}\n")

# # 5) Peek at the ones missing text
# print("IDs with missing text:")
# print(subset[subset["ArticleText"].str.strip() == ""]["ArticleID"].tolist())

# # 6) And sample of those that did extract:
# print("\nSample extracted text:")
# print(subset[subset["ArticleText"].str.strip() != ""][["ArticleID","ArticleText"]].head(3))


In [ ]:
import pandas as pd

# 1. Load your CSV
df = pd.read_csv("soros_text.csv", dtype=str).fillna("")

# 2. Create a boolean mask for “has any non‑whitespace text”
has_text = df["ArticleText"].str.strip() != ""

# 3. Counts
num_with    = has_text.sum()
num_without = len(df) - num_with

print(f"Articles WITH text   : {num_with}")
print(f"Articles WITHOUT text: {num_without}")

# 4. List of ArticleIDs with empty ArticleText
empty_ids = df.loc[~has_text, "ArticleID"].tolist()
print("\nArticleIDs with empty ArticleText:")
print(empty_ids)


In [ ]:
import pandas as pd

# 1) Make sure pandas will show every column
pd.set_option('display.max_columns', None)

# 2) Pick your ArticleID
article_id = "9728994"

# 3) Display the full row (all columns) for that ID
df[df["ArticleID"] == article_id]


In [ ]:
# # 1) Pick your ArticleID
# article_id = "3023148"

# # 2) Extract that row as a Series
# row = df.loc[df["ArticleID"] == article_id].squeeze()

# # 3) Print each column on its own line
# for col, val in row.items():
#     print(f"{col}:\n{val}\n{'-'*40}\n")

### Publications filter and check ArticleID
Kommersant
Literaturnaia gazeta
Nezavisimaia gazeta
Novaia gazeta
Pravda
Slovo
Sovetskaia Rossiia
Trud
Vedomosti
Время МН
Общая газета

In [ ]:
import pandas as pd

# 1) Load your CSV
df = pd.read_csv("soros_text.csv", dtype=str).fillna("")

# 2) Define the list of publications you care about
pubs = [
    "Kommersant", "Literaturnaia gazeta", "Nezavisimaia gazeta", "Novaia gazeta",
    "Pravda", "Slovo", "Sovetskaia Rossiia", "Trud", "Vedomosti",
    "Время МН", "Общая газета"
]

# 3) Filter down to just those rows
sub = df[df["Publication"].isin(pubs)].copy()

# 4) Create a boolean column for “has text”
sub["has_text"] = sub["ArticleText"].str.strip() != ""

# 5) Group and aggregate
result = (
    sub
    .groupby("Publication")["has_text"]
    .agg(
        with_text = lambda x: x.sum(),
        without_text = lambda x: (~x).sum()
    )
    .reindex(pubs)   # keep the original order
)

result


In [ ]:
# 1) Get unique, non-null values
unique_pubs = df["Publication"].dropna().unique()

# 2) Print them as a list
# print(list(unique_pubs))

for pub in unique_pubs:
    print(pub)

In [ ]:
# How many match list
pubs = [
    "Kommersant", "Literaturnaia gazeta", "Nezavisimaia gazeta",
    "Novaia gazeta", "Pravda", "Slovo", "Sovetskaia Rossiia",
    "Trud", "Vedomosti", "Время МН", "Общая газета"
]

# get the distinct Publication values in your df
unique_pubs = set(df["Publication"].dropna().unique())

# find the intersection
matches = unique_pubs & set(pubs)

print(f"Matched publications ({len(matches)} of {len(pubs)}):")
for m in sorted(matches):
    print(" •", m)


In [ ]:
# Find publications
# Your target publications
pubs = [
    "Kommersant", "Literaturnaia gazeta", "Nezavisimaia gazeta",
    "Novaia gazeta", "Pravda", "Slovo", "Sovetskaia Rossiia",
    "Trud", "Vedomosti", "Время МН", "Общая газета"
]

# Boolean mask for “has any non‑blank text”
has_text = df["ArticleText"].fillna("").str.strip() != ""

for pub in pubs:
    sub = df[df["Publication"] == pub]
    non_null_ids = sub.loc[has_text & (df["Publication"] == pub), "ArticleID"].tolist()
    null_ids     = sub.loc[~has_text & (df["Publication"] == pub), "ArticleID"].tolist()

    print(f"=== {pub} ===")
    print(f"With text    ({len(non_null_ids)}): {non_null_ids}")
    print(f"Without text ({len(null_ids)    }): {null_ids}\n")
